# Ionizing radiation burst detection in Tuna-9

**Authors:** Marios Samiotis (m.samiotis@tudelft.nl)

**Date:** December 15, 2025

# 1. Introduction

First, we will run the following cell in order to login to the Quantum Inspire platform. You will need an account in order to login to the platform.

In [ ]:
! qi login "https://staging.qi2.quantum-inspire.com"

In [ ]:
import numpy as np
import time
from qiskit import QuantumCircuit
from qiskit_quantuminspire.qi_provider import QIProvider
from qi_utilities.utility_functions.result_processing import return_raw_data
from tqdm import tqdm

# 2. Connect to the Quantum Inspire backend Tuna-9 in the staging environment

We will first need to connect to the Quantum Inspire provider, by running the following cell,

In [ ]:
provider = QIProvider()

You may see the full list of all available backends of Quantum Inspire platform by running

In [ ]:
provider.backends()

We now connect to the backend Tuna-9 backend by running the following cell,

In [ ]:
backend_name = "Tuna-9"
backend = provider.get_backend(name=backend_name)

# 3. Creating the detection circuit for Tuna-9

In [ ]:
nr_qubits = 9

X_operation_duration = 0.02 # in [us]
wait_time = 2 # in [us]
readout_duration = 0.800 # in [us]
cycle_time = 10 # in [us]
idle_time = cycle_time - (X_operation_duration + wait_time + readout_duration)

nr_measurements_per_circuit = 2**7
nr_circuit_runs = 50 * 2**2
job_limit = 10000

In [ ]:
qc = QuantumCircuit(nr_qubits, nr_qubits * nr_measurements_per_circuit)

CC_cycle_time = 0.02 # in [us]


for repetition in range(nr_measurements_per_circuit):
    for qubit_idx in range(nr_qubits):
        qc.x(qubit_idx)
    qc.barrier()
    for qubit_idx in range(nr_qubits):
        qc.delay(wait_time / CC_cycle_time, qubit_idx, unit='dt')
    qc.barrier()
    for qubit_idx in range(nr_qubits):
        qc.measure(qubit = qubit_idx, cbit = qubit_idx + repetition*nr_qubits)
    qc.barrier()
    for qubit_idx in range(nr_qubits):
        qc.delay(idle_time / CC_cycle_time, qubit_idx, unit='dt')
    qc.barrier()

In [ ]:
from datetime import datetime
import h5py
import json
import os

def generate_hdf5_file(raw_data_list):

    data_directory = r'/Users/mariossamiotis/Documents/Tuna9_cosmic_ray_data/'
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    timestamp = timestamp.split('_')

    date_dir = os.path.join(data_directory, f"{timestamp[0]}")
    if os.path.isdir(date_dir):
        pass
    else:
        os.makedirs(date_dir, exist_ok=False)

    job_raw_data = []

    for circuit_shot_idx in range(len(raw_data_list)):
        ordered_shot_results = raw_data_list[circuit_shot_idx][::-1]
        
        for mst_block_idx in range(nr_measurements_per_circuit):
            measurement_list = [127]
            for qubit_idx in range(nr_qubits):
                measurement_list.append(ordered_shot_results[qubit_idx + mst_block_idx * nr_qubits])
            job_raw_data.append(measurement_list)

    job_raw_data = np.array(job_raw_data, dtype=np.int8)

    job_dir = os.path.join(date_dir, f"{timestamp[1]}_XOR_{int(cycle_time)}_run")
    os.mkdir(job_dir)

    hdf5_file_dir = os.path.join(job_dir, f"{timestamp[1]}_XOR_{int(cycle_time)}_run.hdf5")
    with h5py.File(hdf5_file_dir, 'w') as file:
        file.create_dataset('Experimental Data/Data', data=job_raw_data, compression="gzip")

    metadata_dict = {}
    metadata_dict['data_timestamp'] = f"{timestamp[0]}_{timestamp[1]}"
    metadata_dict['backend_name'] = backend_name
    metadata_dict['nr_qubits'] = nr_qubits
    metadata_dict['X_operation_duration [us]'] = X_operation_duration
    metadata_dict['wait_time [us]'] = wait_time
    metadata_dict['readout_duration [us]'] = readout_duration
    metadata_dict['cycle_time [us]'] = cycle_time
    metadata_dict['idle_time [us]'] = idle_time
    metadata_dict['nr_measurements_per_circuit'] = nr_measurements_per_circuit
    metadata_dict['nr_circuit_runs'] = nr_circuit_runs
    metadata_dict['job_limit'] = job_limit

    json_path = f"{job_dir}/{timestamp[0]}_{timestamp[1]}_metadata.json"
    with open(json_path, 'w') as file:
        json.dump(metadata_dict, file, indent=3)

# 4. Run the job

In [ ]:
for job_idx in tqdm(range(job_limit)):
    try:
        job = backend.run(qc, shots=nr_circuit_runs, memory = True)
        result = job.result(timeout = 600)
        raw_data_list = return_raw_data(qc, result)
        
        generate_hdf5_file(raw_data_list)
    except:
        print('Processor seems to be offline!')
        time.sleep(600)